# EDGAR API Testing and Analysis
This notebook demonstrates how to interact with the SEC's EDGAR API to fetch and analyze company filings.

## Import Required Libraries
We'll import the necessary libraries for API interaction, data manipulation, and testing.

In [ ]:
import requests
import pandas as pd
import unittest
import json
from datetime import datetime
import logging

## Set Up EDGAR API Connection
Configure the base URL and required headers for EDGAR API interactions.

In [ ]:
# EDGAR API Configuration
BASE_URL = "https://data.sec.gov/api/xbrl/companyfacts"
HEADERS = {
    "User-Agent": "Your Name your.email@domain.com",
    "Accept": "application/json"
}

def create_edgar_session():
    session = requests.Session()
    session.headers.update(HEADERS)
    return session

## Fetch Company Filings
Create functions to fetch and handle company filing data from EDGAR.

In [ ]:
def fetch_company_filings(cik, session=None):
    """
    Fetch company filings for a given CIK number
    """
    if session is None:
        session = create_edgar_session()
    
    try:
        response = session.get(f"{BASE_URL}/CIK{cik:010d}")
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data for CIK {cik}: {str(e)}")
        return None

## Parse and Analyze Filing Data
Transform the raw filing data into a structured format for analysis.

In [ ]:
def parse_filing_data(filing_data):
    """
    Parse and structure filing data into a pandas DataFrame
    """
    if not filing_data or 'facts' not in filing_data:
        return pd.DataFrame()
    
    records = []
    for concept, data in filing_data['facts'].get('us-gaap', {}).items():
        for unit, values in data.get('units', {}).items():
            for value in values:
                record = {
                    'concept': concept,
                    'unit': unit,
                    'value': value.get('val'),
                    'filed_date': value.get('filed'),
                    'frame': value.get('frame', '')
                }
                records.append(record)
    
    return pd.DataFrame(records)

## Write Unit Tests for EDGAR API
Create unit tests to verify the API functionality.

In [ ]:
class TestEDGARAPI(unittest.TestCase):
    def setUp(self):
        self.session = create_edgar_session()
    
    def test_api_connection(self):
        """Test basic API connectivity"""
        # Test with Apple Inc. (CIK: 320193)
        response = fetch_company_filings(320193, self.session)
        self.assertIsNotNone(response)
        self.assertIn('facts', response)
    
    def test_data_parsing(self):
        """Test parsing of filing data"""
        sample_data = {
            'facts': {
                'us-gaap': {
                    'Revenue': {
                        'units': {
                            'USD': [
                                {'val': 1000, 'filed': '2023-01-01', 'frame': '2023Q1'}
                            ]
                        }
                    }
                }
            }
        }
        df = parse_filing_data(sample_data)
        self.assertFalse(df.empty)
        self.assertEqual(len(df), 1)

if __name__ == '__main__':
    unittest.main(argv=['ignored', '-v'])